Find the description here:  https://gerardnico.com/data_mining/stepwise_regression

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from scipy import stats
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
def convertime(datetime_str):
    return datetime.strptime(datetime_str, "%Y%m%dT000000").timestamp()

Exploration of train dataset

In [2]:
train = pd.read_csv('kc_house_train_data.csv')
train.head()

id             date     price  bedrooms  bathrooms  sqft_living  \
0  7129300520  20141013T000000  221900.0         3       1.00         1180   
1  6414100192  20141209T000000  538000.0         3       2.25         2570   
2  5631500400  20150225T000000  180000.0         2       1.00          770   
3  2487200875  20141209T000000  604000.0         4       3.00         1960   
4  1954400510  20150218T000000  510000.0         3       2.00         1680   

   sqft_lot  floors  waterfront  view  ...  grade  sqft_above  sqft_basement  \
0      5650     1.0           0     0  ...      7        1180              0   
1      7242     2.0           0     0  ...      7        2170            400   
2     10000     1.0           0     0  ...      6         770              0   
3      5000     1.0           0     0  ...      7        1050            910   
4      8080     1.0           0     0  ...      8        1680              0   

   yr_built  yr_renovated  zipcode      lat     long  sqft_living15  \
0      1955             0    98178  47.5112 -122.257           1340   
1      1951          1991    98125  47.7210 -122.319           1690   
2      1933             0    98028  47.7379 -122.233           2720   
3      1965             0    98136  47.5208 -122.393           1360   
4      1987             0    98074  47.6168 -122.045           1800   

   sqft_lot15  
0        5650  
1        7639  
2        8062  
3        5000  
4        7503  

[5 rows x 21 columns]

In [3]:
train.shape

(17384, 21)

In [4]:
train.columns

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15'],
      dtype='object')

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17384 entries, 0 to 17383
Data columns (total 21 columns):
id               17384 non-null int64
date             17384 non-null object
price            17384 non-null float64
bedrooms         17384 non-null int64
bathrooms        17384 non-null float64
sqft_living      17384 non-null int64
sqft_lot         17384 non-null int64
floors           17384 non-null float64
waterfront       17384 non-null int64
view             17384 non-null int64
condition        17384 non-null int64
grade            17384 non-null int64
sqft_above       17384 non-null int64
sqft_basement    17384 non-null int64
yr_built         17384 non-null int64
yr_renovated     17384 non-null int64
zipcode          17384 non-null int64
lat              17384 non-null float64
long             17384 non-null float64
sqft_living15    17384 non-null int64
sqft_lot15       17384 non-null int64
dtypes: float64(5), int64(15), object(1)
memory usage: 2.8+ MB


In [6]:
train.describe()

id         price      bedrooms     bathrooms   sqft_living  \
count  1.738400e+04  1.738400e+04  17384.000000  17384.000000  17384.000000   
mean   4.574349e+09  5.393666e+05      3.369363      2.115048   2080.029510   
std    2.872356e+09  3.696912e+05      0.906468      0.771783    921.630888   
min    1.000102e+06  7.500000e+04      0.000000      0.000000    290.000000   
25%    2.124087e+09  3.200000e+05      3.000000      1.750000   1420.000000   
50%    3.892800e+09  4.500000e+05      3.000000      2.250000   1910.000000   
75%    7.304301e+09  6.400000e+05      4.000000      2.500000   2550.000000   
max    9.900000e+09  7.700000e+06     10.000000      8.000000  13540.000000   

           sqft_lot        floors    waterfront          view     condition  \
count  1.738400e+04  17384.000000  17384.000000  17384.000000  17384.000000   
mean   1.509191e+04      1.494248      0.007651      0.236079      3.410780   
std    4.145927e+04      0.539443      0.087136      0.768008      0.649792   
min    5.200000e+02      1.000000      0.000000      0.000000      1.000000   
25%    5.049500e+03      1.000000      0.000000      0.000000      3.000000   
50%    7.616000e+03      1.500000      0.000000      0.000000      3.000000   
75%    1.066525e+04      2.000000      0.000000      0.000000      4.000000   
max    1.651359e+06      3.500000      1.000000      4.000000      5.000000   

              grade    sqft_above  sqft_basement      yr_built  yr_renovated  \
count  17384.000000  17384.000000   17384.000000  17384.000000  17384.000000   
mean       7.655028   1787.844512     292.184998   1971.152727     83.107973   
std        1.169818    827.107595     444.404136     29.328722    398.692283   
min        1.000000    290.000000       0.000000   1900.000000      0.000000   
25%        7.000000   1200.000000       0.000000   1952.000000      0.000000   
50%        7.000000   1560.000000       0.000000   1975.000000      0.000000   
75%        8.000000   2210.000000     560.000000   1997.000000      0.000000   
max       13.000000   9410.000000    4820.000000   2015.000000   2015.000000   

            zipcode           lat          long  sqft_living15     sqft_lot15  
count  17384.000000  17384.000000  17384.000000   17384.000000   17384.000000  
mean   98077.936896     47.559313   -122.213281    1985.994995   12776.380867  
std       53.525617      0.138703      0.140906     686.512835   27175.730523  
min    98001.000000     47.159300   -122.519000     399.000000     651.000000  
25%    98033.000000     47.468650   -122.328000    1490.000000    5100.000000  
50%    98065.000000     47.571400   -122.229000    1840.000000    7620.000000  
75%    98117.000000     47.677625   -122.125000    2360.000000   10065.250000  
max    98199.000000     47.777600   -121.315000    6210.000000  871200.000000

In [7]:
print("feature"," unique values")
for feature in train.columns:
   

    print(feature,"                 ",train[feature].value_counts().count())
print("date","                 ",train['date'].value_counts().count())

feature  unique values
id                   17273
date                   368
price                   3552
bedrooms                   11
bathrooms                   29
sqft_living                   957
sqft_lot                   8439
floors                   6
waterfront                   2
view                   5
condition                   5
grade                   12
sqft_above                   869
sqft_basement                   295
yr_built                   116
yr_renovated                   70
zipcode                   70
lat                   4864
long                   729
sqft_living15                   717
sqft_lot15                   7571
date                   368


In [8]:
train
train.shape

(17384, 21)

In [9]:
train['date'] = train['date'].apply(convertime)
train = train.drop(columns=['id'])
train_x = train.drop(columns=['price'])
train_y = train.price

In [10]:
train_y.shape

(17384,)

Exploration for test dataset

In [11]:
test = pd.read_csv('kc_house_test_data.csv')
test.head()
test['date'] = test['date'].apply(convertime)

In [12]:
test = test.drop(columns=['id'])
test_x = test.drop(columns=['price'])
test_y = test.price

In [13]:
test_y.shape

(4229,)

In [14]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(train_x.to_numpy())
scaler.transform(train_x)
scaler.transform(test_x)

array([[-1.36609463, -0.40748647, -1.44481169, ..., -0.11155904,
        -0.30006856, -0.0029211 ],
       [ 0.76862898,  0.69572873,  1.14666648, ..., -1.147741  ,
         0.22433584, -0.32295851],
       [ 0.04229149, -0.40748647, -0.14907261, ..., -0.03349053,
        -1.39257774, -0.29701549],
       ...,
       [-0.134864  ,  0.69572873,  0.49879694, ...,  0.32846344,
         0.77787382, -0.24851494],
       [ 1.03436221,  0.69572873,  0.49879694, ..., -1.05547822,
        -0.22723462, -0.20520299],
       [-1.13579249, -1.51070166, -1.76874647, ..., -0.60835861,
        -1.40714453, -0.39629811]])

In [ ]:
# rvs1 = stats.norm.rvs(loc=5,scale=10,size=500)
# rvs2 = stats.norm.rvs(loc=5,scale=10,size=500)
# stats.ttest_ind(rvs1,rvs2)

stats.ttest_ind(np.array(train_y),np.array(train_x.sqft_lot), equal_var = False)


In [16]:
x_train, x_validation, y_train, y_validation = train_test_split(train_x, train_y, test_size=0.33, random_state=42)


In [23]:
x_train.iloc[3]

date             1.416422e+09
bedrooms         2.000000e+00
bathrooms        2.000000e+00
sqft_living      1.350000e+03
sqft_lot         1.209000e+03
floors           3.000000e+00
waterfront       0.000000e+00
view             0.000000e+00
condition        3.000000e+00
grade            8.000000e+00
sqft_above       1.350000e+03
sqft_basement    0.000000e+00
yr_built         2.013000e+03
yr_renovated     0.000000e+00
zipcode          9.819900e+04
lat              4.764900e+01
long            -1.223830e+02
sqft_living15    1.310000e+03
sqft_lot15       9.820000e+02
Name: 16010, dtype: float64

In [ ]:

def train(train_x,validation_x, train_y,  validation_y):
    param_dict= dict.fromkeys(train_x.columns.to_numpy(), 0)
    param_dict['intercept'] = train_y.mean()
    number_of_features = (train_x.iloc[0].size)
    number_of_samples =  train_x['bedrooms'].size
    stop  = False
    best_score = -10000000
    feature_set = (train_x.columns.to_numpy())
    residual = train_y.to_numpy()-np.ones(number_of_samples)*train_y.mean()
#     print(len(residual))
    model_features = []
    while True:
        print(len(feature_set))
        if feature_set.size ==0:
            return param_dict
        scores = []
        models = []
        features_to_train = model_features
        for feature in feature_set:
            
            features_to_train.append(feature)
            model1 = LinearRegression(fit_intercept=True)
#             print("length",len(residual))
            model1.fit(np.array(np.matrix(train_x.loc[ : , features_to_train ].to_numpy()).T),residual)
            Y_pred = model1.predict(np.array(np.matrix(validation_x.loc[ : , features_to_train ].to_numpy()).T))
            models.append(model1)
            a=mean_squared_error(validation_y.to_numpy(),Y_pred)
            scores.append(a)
        print(scores)
        min_score_feature = np.argmin(scores)
        min_score = np.min(scores)
        best_model = models[min_score_feature]
        best_param = best_model.coef_
        param_dict[feature_set[min_score_feature]] = best_param[0]
        residual = residual -np.ones(number_of_samples)*best_param[0]
        print("score", max_score)
        if (max_score -best_score)<.0005 :
            return param_dict
        best_score = max_score
        print(max_score_feature)   
        feature_set = np.delete(feature_set,max_score_feature)
    return param_dict
            
        

In [ ]:
train(x_train, x_validation, y_train, y_validation )

In [ ]:
Python
# Select only 2 columns from dataFrame and create a new subset DataFrame
columnsData = dfObj.loc[ : , ['Age', 'Name'] ]
1
2
# Select only 2 columns from dataFrame and create a new subset DataFrame
columnsData = dfObj.loc[ : , ['Age', 'Name'] ]

In [24]:
a = [2,3,4,5,2]
b = a
print(b)

[2, 3, 4, 5, 2]


In [ ]:
rvs3 = stats.norm.rvs(loc=5, scale=20, size=500)
stats.ttest_ind(rvs1, rvs3)

stats.ttest_ind(rvs1, rvs3, equal_var = False)
print(type(rvs3))

In [ ]:
train_y.to_numpy()-np.ones(17384)*train_y.mean()

In [ ]:
# Function to remove elements using discard() 
def Remove(sets): 
    sets.discard(21) 
    print (sets) 
      
# Driver Code 
sets = set([10, 20, 26, 41, 54, 20]) 
print(sets)
Remove(sets) 

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
X = np.array([2,3 4, 5, 9,7])
# y = 1 * x_0 + 2 * x_1 + 3
y = np.array([2,4 ,5,8,9,1])
reg = LinearRegression().fit(X, y)
reg.score(X, y)

print(reg.coef_)

print(reg.intercept_)

print(reg.predict(np.array([[3, 5]])))


In [ ]:
a = [2,4]
a = dict.fromkeys(train.columns.to_numpy(), 0)
a['intercept'] = 9
print(a)

In [ ]:
X = np.array([2,3, 4, 5, 9,7])
np.array(np.matrix(X).T)